## Preprocessing the data

Create a TimeSeries object from a Pandas DataFrame, and split it in train/validation series:

In [6]:
import pandas as pd
from darts import TimeSeries

# Read a pandas DataFrame 2023
load2023 = pd.read_csv("Load_Data/Total Load - Day Ahead _ Actual_2023.csv", delimiter=",")

# Change the timestamp following the dataseries for DARTS
load2023['Time (CET/CEST)'] = load2023['Time (CET/CEST)'].str.split(' - ').str[0]
# Delete the initial day ahead forecast data
load2023 = load2023.drop('Day-ahead Total Load Forecast [MW] - BZN|SE3',axis=1)   
# Change the name to simple name
load2023 = load2023.rename(columns={'Time (CET/CEST)': 'completetime', 'Actual Total Load [MW] - BZN|SE3': 'Load [MW]'})
# Convert 'time' to datetime for easier splitting
load2023['datetime'] = pd.to_datetime(load2023['completetime'], format='%d.%m.%Y %H:%M')
# Create 'date' and 'time' columns
load2023['date'] = load2023['datetime'].dt.strftime('%Y-%m-%d')
load2023['time'] = load2023['datetime'].dt.strftime('%H:%M:%S')
load2023 = load2023.drop(['completetime', 'datetime'], axis=1)
# Reorder the column
load2023 = load2023[['date', 'time', 'Load [MW]']]

print(load2023)



            date      time  Load [MW]
0     2023-01-01  00:00:00     8955.0
1     2023-01-01  01:00:00     8943.0
2     2023-01-01  02:00:00     8929.0
3     2023-01-01  03:00:00     8887.0
4     2023-01-01  04:00:00     8859.0
...          ...       ...        ...
8756  2023-12-31  19:00:00    11947.0
8757  2023-12-31  20:00:00    11561.0
8758  2023-12-31  21:00:00    11276.0
8759  2023-12-31  22:00:00    11035.0
8760  2023-12-31  23:00:00    10831.0

[8761 rows x 3 columns]
